## Definindo bibliotecas

Definição das bibliotecas necessárias para o projeto:

1. Pandas: responsável pela manipulação das planilhas em excel (ou dataframes);
2. Numpy: responsável pela manipulação matemática dentro dos dataframes;
3. Datetime: responsável pela manipulação de datas;
4. OS: responsável pela manipulação de recursos do windows;

In [2]:
import pandas as pd
import numpy as np
import datetime as dt
import os

## Parametrizando usuário

Através da biblioteca OS a função .getlogin() retorna o usuário do computador que está rodando o código.

In [3]:
user = os.getlogin()
user

'mayaram4rtins'

## Definindo diretório dos arquivos em excel

A função .path.join da biblioteca OS irá retornar o caminho do diretório onde estarão os arquivos excel que serão lidos.

In [4]:
dir1 = os.path.join(user, 'Documentos', 'compilador-de-arquivos-excel')
dir1

'mayaram4rtins\\Documentos\\compilador-de-arquivos-excel'

## Tratamento de dados dentro dos arquivos primários

### Abertura do excel em um dataframe

Através do pandas, formar um dataframe com o arquivo que será modificado

In [5]:
arq_df = pd.read_excel('Entradas_Saidas_Betofarma.xlsx', engine='openpyxl')
display(arq_df)

,Registro,Formulário,Tipo E/S,Empresa Contábil,Empresa,Movimento,Emissão,NF Número,ECF Caixa,ECF Série,...,CFOp,ICMS CST,Valor Contábil,ICMS Base Cálc.,ICMS Aliq.,ICMS Crédito / Débito,ICMS Isento,ICMS Outros,ICMS ST Valor,Observações
0,18448,Vendas PDV,S,4,4,2022-01-01,2022-01-01,18448,6.0,871652.0,...,5405,60,34.20,0.00,0.0,0.00,0.0,34.20,0,NaN
1,18449,Vendas PDV,S,4,4,2022-01-01,2022-01-01,18449,6.0,871652.0,...,5405,60,19.99,0.00,0.0,0.00,0.0,19.99,0,NaN
2,18450,Vendas PDV,S,4,4,2022-01-01,2022-01-01,18450,6.0,871652.0,...,5405,60,20.99,0.00,0.0,0.00,0.0,20.99,0,NaN
3,18451,Vendas PDV,S,4,4,2022-01-01,2022-01-01,18451,6.0,871652.0,...,5405,60,20.99,0.00,0.0,0.00,0.0,20.99,0,NaN
4,18452,Vendas PDV,S,4,4,2022-01-01,2022-01-01,18452,6.0,871652.0,...,5405,60,13.36,0.00,0.0,0.00,0.0,13.36,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16259,304917,Vendas PDV,S,4,4,2022-01-31,2022-01-31,304917,2.0,278839.0,...,5405,60,13.90,0.00,0.0,0.00,0.0,13.90,0,NaN
16260,304918,Vendas PDV,S,4,4,2022-01-31,2022-01-31,304918,2.0,278839.0,...,5405,60,19.97,0.00,0.0,0.00,0.0,19.97,0,NaN
16261,304919,Vendas PDV,S,4,4,2022-01-31,2022-01-31,304919,2.0,278839.0,...,5405,60,13.89,0.00,0.0,0.00,0.0,13.89,0,NaN
16262,1143313,Nota Fiscal de Compra e Outras Entradas,E,4,4,2022-01-31,2022-01-27,647725,NaN,NaN,...,1102,0,884.69,884.69,18.0,159.17,0.0,0.00,0,NaN


### Manipulando a data da coluna movimento

É necessário que o formato final da data nessa coluna seja: d/m/y

In [6]:
arq_df['Movimento'] = pd.to_datetime(arq_df['Movimento'], format='%Y-%m-%d', errors='coerce')
arq_df['Movimento'] = arq_df['Movimento'].astype('datetime64[ns]')
arq_df['Movimento'] = arq_df['Movimento'].dt.strftime("%d/%m/%Y")
arq_df['Movimento']

0        01/01/2022
1        01/01/2022
2        01/01/2022
3        01/01/2022
4        01/01/2022
            ...    
16259    31/01/2022
16260    31/01/2022
16261    31/01/2022
16262    31/01/2022
16263    31/01/2022
Name: Movimento, Length: 16264, dtype: object

### Soma de formulário com nomes vazios

Somar valor contábil e ICMS crédito/débito que são de "Formulário: Venda PDV" na mesma data de "Movimento" e da mesma "Empresa"

Alterando a coluna de movimento para formatação de data que será utilizado posteriormente

In [1]:
vendas_pdv_df = arq_df.loc[arq_df['Formulário'] == 'Vendas PDV',
                                 ["Formulário", "Tipo E/S", "Movimento", "Empresa", "Valor Contábil", 
                                  "ICMS Crédito / Débito"]]

display(vendas_pdv_df)

vendas_pdv_3_df = vendas_pdv_df.groupby(['Formulário',
                                         'Tipo E/S',
                                         'Empresa',
                                         'Movimento'])[['Valor Contábil', 'ICMS Crédito / Débito']].sum().reset_index()

display(vendas_pdv_3_df)

NameError: name 'vendas_pdv_df' is not defined

## Construção do arquivo final

### Condição para arquivo final

#### dePara_VlrContábil_e_ICMS

Vendas PDV: contábil credito: 319, contabil debito 37
ICMS s/ Vendas PDV: contabil credito: 243, contabil debito 327
PIS s/ Vendas PDV: credito 246, debito 329
COFINS s/ Vendas PDV: credito 241, debito 326
Nota de Devolução de Compras + NF e coluna numero + coluna nome: credito 80, debito 204
    
Digitação de NF de Faturamento + NF e coluna numero + coluna nome (saída):  
    Se CFOPS 5927, debito 592, credito 80
    Se CFOPS diferente de 5927 mas o NOME é DrogaEx, credito 80, debito 92
    Se CFOPS diferente de tudo isso, credito 341, debito 144
    
ICMS s/ Digitação de NF de Faturamento + NF e coluna numero + coluna nome (saída): credito 243, debito 347

Digitação de NF de Faturamento + NF e coluna numero + coluna nome (entrada):  
    Se o NOME é DrogaEx, credito 93, debito 80
    Se o NOME diferente de DrogaEX, credito 10000, debito 20000
    
ICMS s/ Digitação de NF de Faturamento + NF e coluna numero + coluna nome (entradas): credito 80, debito 59

Devolução de Produtos Caixas + NF e coluna numero + coluna nome: credito 41, debito 332
ICMS s/ Devolução de Produtos Caixas + NF e coluna numero + coluna nome: credito 80, debito 59

Nota Fiscal de Compra e Outras Entradas + NF e coluna numero + coluna nome: 
    Se o NOME é DBR ou conter, credito 10055 e debito 374
    Se o NOME é ou contém BETOFARMA, DELMAR, DOTTO, DEMAC, DROGAEX, EX NG, POA, METROFARMA,    MIYAFARMA, NOVA CAIEIRAS e DROGAROMERO, credito 217 e debito 80
    Se o NOME é diferente disso tudo, credito 204, debito 80
    
ICMS s/ Nota Fiscal de Compra e Outras Entradas + NF e coluna numero + coluna nome: credito 80, debito 59

#### dePara_PIS_e_COFINS

Tudo que for TIPO E/S: E, PIS (debito 63 e credito 80) e COFINS (debito 57 e credito 80)

Tudo que for TIPO E/S: S, PIS (debito 63 e credito 80) e COFINS (debito 57 e credito 80)
    PIS: Digitação de NF de Faturamento + NF e coluna numero + coluna nome (saída): credito 246, debito 349
    COFINS: Digitação de NF de Faturamento + NF e coluna numero + coluna nome (saída): credito 241, debito 346